## **Hello!**

Here are the data cleaning, pre-processing, feature engineering and modeling steps of our proposed solution. We did our best to explicitate our process, assumptions and decisions while remaining a concise as possible. As a whole, the process was iterative and was preceded by an in-depth exploratory data analysis.

Thank you for your time in evaluating our submission!

Cynthia & Juliette

## **1. Data cleaning**

- **Removing redundant variables**: we dropped columns that provided overlapping information. For, 'GoingID' and 'GoingAbbrev' columns were removed as they capture the same information as the 'Wetness' variable.

- **Type casting**: we performed type casting on certain columns (e.g. IDs, date variables, Saddlecloth, WeightCarried). This ensured that numerical and categorical data are treated appropriately in subsequent manipulations.

- **Updating values**: to enhance the clarity of the data and our own understanding, we updated values in some columns (e.g. RacingSubType and SexRestriction). We cleaned AgeRestriction by splitting it into 2 variables: min_age (minimum authorized age) and max_age (maximum authorized age), using 2 as a lower bound and 16 as an upper bound when unspecified. Values for HandicapDistance were unified by removing minus signs and due to the very low number of "75" values, these instances were replaced by 50.

- **Missing values**: For columns where there were unrecognized missing values (e.g. empty strings), we assigned them to either NaN or a labelled "None"/"Other" category where appropriate. Several columns had "0" values when they were non-applicable, in which case they were replaced with NaN (e.g. Barrier for volte start races). For "RaceOverallTime", 0 values were interpreted as missing data and were assigned as such. Other variables had abnormal values that we interpreted as input errors or missing values (e.g. -9 or 999 in WideOffRail and BeatenMargin, respectively).

## **2. Feature engineering**

### **Race variables**

- **Nb_Participants variable**
  - Rationale: races have varying numbers of participants, making the direct comparison of finish positions unfair from one race to another. This variable indicates how many horses participate in each unique race, and is used for further feature engineering.
  - Method: counts the number of HorseIDs for each RaceID.


- **Season variable**
  - Rationale: We hypothesized the time of year when a race takes place may affect racing conditions and horse performance due to weather patterns, track condition, and seasonal horse fitness levels.
  - Method: The 'Season' variable is derived from the 'RaceStartTime' by mapping months to seasons.


- **Relative_age variable**
  - Rationale: The age of a horse relative to its competitors can be a factor in its performance, with younger horses potentially being less experienced and older horses possibly past their peak.
  - Model Impact: The 'Relative_age' variable contextualizes a horse's age within the specific race field, potentially identifying advantageous age-related performance trends.
  - Method: This variable scales horse ages for each RaceID, with 0 representing the youngest horse and 1 the oldest horse in the race.

### **Race outcome variables**

- **Started_race and Finished_race variables**
  - Rationale: Tracking whether a participant starts and finishes races can signal its reliability and potential issues that may affect its future performance.
  - Model Impact: The 'Started_race' and 'Finished_race' binary variables help differentiate between horses that consistently compete and those that may be less "safe bets".
  - Method: The 'Started_race' variable is true for all horses not marked with 'NP' in 'FinishPosition', while 'Finished_race' is true for horses that have a numerical finish position and were not disqualified, thus capturing participation and completion rates.

- **Placed variable**
  - Rationale: the 'Placed' variable considers horses in the top 7 positions, which typically receive prize money, thus highlighting competitors that are successful despite not being first, and adding more granularity to our assessments of performance compared to the binary WIN variable.
  - Method: This variable is true for horses finishing in positions 1-7 that were not disqualified.

- **Updated FinishPosition variable**
  - This variable is entirely numerical, assigning horses that did not complete the race with the last position (the last position being equal to the number of participants "Nb_Participants")

- **WIN Variable**
  - Rationale: In order to find the winner, we found that Prizemoney was a better indicator than using FinishPosition, because of the possibility of disqualification (some horses have FinishPosition=1 but Disqualified=True and Prizemoney=0).
  - Model Impact: This is our target variable, and serves as ground truth for our model.
  - Method: The winner is identified for each race by selecting the row with the highest 'Prizemoney' amongst each RaceID.

### Past performance variables

- **Performance Score**
  - Rationale: A performance score based on finish position and accounting for the number of participants offers a quantitative measure of performance that can be compared across races and participants.
  - Model Impact: this performance score reflects each horse's placement relative to others, offering a more granular assessment of performance.
  - Method: We first updated the FinishPosition variable by adding 1 to horses who either did not start / did not finish the race, to differentiate them from horses who finished last but did complete the race. The performance score is calculated as the reciprocal of the updated 'FinishPosition' and then min-max scaled within each RaceID.

- **Cumulative past performance variables for horses, trainers, and jockeys**
  - '**Nb_Participations**':
    - Rationale: indicator of experience level
    - Method: cumulative sum of past participations for horses, trainers, and jockeys
  - '**Nb_of_Wins**':
    - Rationale: Wins are a direct indicator of success and potential for future winnings.
    - Method: cumulative number of wins in past races for horses, trainers, and jockeys
  - '**Winrate**':
    - Rationale: balances the number of past wins with the number of past participations
    - Method: Nb_of_Wins / Nb_Participations
  - '**Nb_of_times_Placed**':
    - Rationale: provides more flexibility compared to only considering wins, which can help identify strong performers who are likely to place well
    - Method: cumulative count of finishing a race in the top 7
  - '**Total_Prizemoney**':
    - Rationale: prize money takes into account the importance and level of the race in addition to the performance of the participants.
    - Method: cumulative amount of $ won
  - '**Cumul_Score**': cumulative score from past races
  - '**AvgScore_All**': cumulative average score
  - '**AvgScore_Last5**':
    - Rationale: recent form can be a more important indicator for the upcoming race, rather than considering the entirety of a horse's "career". These variables provide the model with more immediate temporal trends
    - Method: average score of the last 5 races

- **Past performance scores for horses, trainers and jockeys**
  - By dividing the cumulated normalized scores by the number of past participations (for horses, jockeys and trainers, respectively), we obtain scores or "ranks" for each entity. Once scaled at the RaceID level, these features indicate where a participant is placed compared to competitors in terms of past performances.

- **Combined Performance Score**
  - Rationale: Combining the past performance score of horse, jockey, and trainer can give us a fuller picture of where they rank as a team.
  - Model Impact: A combined performance score can quantify the synergy of the three parties, offering a holistic view of the team.
  - Method: We compute 3 different versions of combined performance scores by multiplying the win rates, average scores, or last 5 race scores of the horse, jockey, and trainer.
  - We also obtain a composite score, where the scores of the horse, trainer and jockey are weighed differently. The weights were selected based on feature importance scores of past iterations of the model.

- **Consistency of Horse's Finish Position**
  - Rationale: A horse that consistently finishes in top positions is likely to have better training, conditioning, and inherent ability, making it a strong contender in future races and a safer "bet".
  - Model Impact: By quantifying finish position consistency the model can better assess the reliability of a horse's performance. This feature can help distinguish consistently good performers from horses that might have had a few lucky wins but are inconsistent.
  - Method: We created 2 new variables: 'horse_consistency_position' takes the standard deviation of the horse's last five finish position, while 'horse_consistency' takes the standard devaition of the horse's last five normalized score.

### Other signals

- **Horse's preferred surface type**
  - Rationale: Based on the provided documentation, we wanted to add a signal of match or mismatch between a race's track surface and the horse's "preference" (based on past performances on similar surfaces).
  - The binary "is_preferred_surface" variable indicates whether the surface is the one the horse tends to perform best on.
  - Method: we use cumulative scores for each surface type divided by the number of past participations, and for each row, identify which of the 3 values is the highest to be selected as the horse's "preferred" type. If the surface of the current race matches that, the "is_preferred_surface" variable is assigned 1.


- **Rest Period**
  - Rationale: The duration of a horse's rest period before a race can greatly affect its performance. Both extended rest and quick turnarounds have implications for a horse's readiness and physical condition.
  - Model Impact: Including the 'Rest Period' as a variable helps to ensure that the model accounts for potential effects of fatigue or well-restedness, which can be particularly influential in a horse's performance.
  - Method: The 'Rest Period' variable is the number of days since a horse's last race, based on the RaceStartTime date variable.


- **Average Beaten Margin**
  - Rationale: The average margin by which a horse is beaten in races can provide insights into how competitive the horse is. A smaller beaten margin could indicate that the horse is often a strong competitor but may have faced some minor issues preventing a win.
  - Model Impact: This feature helps to differentiate between horses that barely miss the win, and those that are often far behind.


- **Field Competitiveness**
  - Rationale: The level of competition in previous races can be a significant factor in interpreting a horse's past performance. Winning a high-stakes race with strong competitors can be more indicative of a horse's ability than winning among a weak field.
  - Model Impact: By incorporating measures of field competitiveness the model can weigh a horse's past wins or places more accurately.
  - Method: We calculated this variable using the 'horse_past_perf_score', 'jockey_past_perf_score', 'trainer_past_perf_score', and 'race_prizemoney_score'. After assigning different weight to these variable, we multiple these variables with the weight and sum them up. We then group the dataset by RaceID and get the average of this score for each race as the 'field_competitiveness'.


- **Pedigree variable** (Dam and Sire Past Performance Scores)
  - Rationale: Pedigree information can be an important indicator of latent potential and performance capacity. A horse's dam (mother) and sire (father) can provide insights into the genetic quality and inherited traits that may influence a horse's capabilities.
  - Model Impact: By quantifying the average performance of a dam's offspring, we can adjust predictions to account for genetic factors.
  - Method: We calculated the 'Dam Performance Score' by taking the average of horse_averagescore_scaled for all offspring of the same dam. This score is then assigned to each horse as an inherited performance metric. We did the same for Sires.


- **Age and Race Prize Money Interaction Term**
  - Rationale: The interaction between a horse's age and the prize money of a race could signal the maturity and experience of a horse relative to the level of competition it has faced
  - Model Impact: This interaction term can help the model to discern whether a horse is in its prime and competing successfully in higher-stakes races, which is often a strong predictor of future performance.
  - Method: We created the interaction term by multiplying the 'HorseAge' variable with 'RacePrizemoney', enabling the model to assess the combined effect of these variables on race outcomes.

## **3. Pre-processing**


**Categorical variables**:
- Binary encoding (0/1) was used for variables with 2 categories
- One-hot encoding was used for variables with 3+ categories.

**Numerical variables**:
- Standard scaling was used on "RacePrizemoney" and "Distance"
- A number of variables were scaled at the RaceID level (using min-max scaling), so that the values reflect where the participant is situated as compared to its competitors, which we expected to be more meaningful in predicting their success.

**Handling missing values in engineered features**:
- For most engineered features, we imputed missing values with 0.5, which was the middleground or neutral placement, given that the values range from 0 to 1. Several imputation values were tested, as well as leaving nulls as NaN, and this choice of value was found to perform best.
- For RestPeriod, horses who were appearing for the first time in the dataset had NaN values, which we decided to impute with the 75th percentile (=27 days) after experimenting.

**Other minor adjustments**:
- For volte start types, the starting line variable is redundant with the handicap distance variable, since it depends on it. For these races, the StartingLine value was replaced with NaN, so that the model could potentially use relationships between StartingLine and performance for mobile start types.
- For mobile start types, the -1 values were assumed to be missing values. As an estimation, we replaced them with line 1 for horses with Saddlecloth between 1 and 9, and with line 2 for Saddlecloth over 9. Despite the number of horses per line varying depending on the track, we could not identify tracks, and therefore used the estimation value of 9 horses per line.
- For attelage races, we replaced WeightCarried 0 values with NaN, as the variable is non-applicable to these races. For the remaining 0 values, they were replaced by the median (of the subset after excluding these 0 values).

## **4. Model training**

- **Train-test split**: The data was split between training (before Nov 1, 2021) and testing set (Nov 1, 2021 and later).
- **Model selection**:
  - We experimented with different classification algorithms. We began with Logistic Regression, using it as a baseline to compare other models' complexity and accuracy. Our other experimentations included Naive Bayes, Decision Trees, Random Forests, Support Vector Machines (SVM) and XGBoost. Models were evaluated for balance between performance and computational demand.
  - We selected **XGBoost**, which outperformed the other algorithms while being computationally feasible on our personal laptops. While SVM and neural network models were considered, their computational requirements exceeded our available resources, which led us to opt for a more efficient model allowing for more iterative experimentations.

- **Feature selection**: For the purpose of the competition, our priority was to optimize the model for the log loss score. Therefore, we decided to retain most engineered features in our final selection instead of opting for minimization (which would have been preferable in other settings). Our experiments repeatedly showed that including this full set of variables led to the best performance. XGBoost's inherent feature selection through regularization allowed us to maintain a comprehensive feature set without manually pruning it. We include a feature importance chart in the notebook to display the contribution of each feature in the model.

- **Parameter tuning**: Given our limited computational capacity, we applied random search with cross-validation to test various parameter combinations. Focusing on important XGBoost parameters like learning rate, tree depth, and subsample size, we aimed for a harmonious balance between the complexity of the model and the avoidance of overfitting.

- By emphasizing the interpretability of our features, we have strived to create a model that provides insights into the factors influencing predictions. This transparency allows for a better understanding of the model's outputs, paving the way for more informed strategic decisions in real-world applications.



## **5. Output post-processing**

- We scale the probabilities obtained through the model as the RaceID level, so that all winprobabilities among a race sum up to 1. The log loss score is computed from this variable.
- From these scaled probabilities, for each unique race we select the highest value and assign it to 1 in "WIN_pred" (predicted label), while all others are set to 0. This variable is used to be compared against the ground truth WIN variable, and obtain other performance metrics (accuracy, precision, recall, and the confusion matrix).

## **6. Performance metrics**

- Accuracy: 0.893
- Precision: 0.295
- Recall: 0.295
- Log loss score:  0.231
- Number of races for which the winner is correctly predicted: 631 out of 2140 (= 29.5% of races)

## **7. Testing our model with new data**

- In order to test our model on new data, we have provided the Final-Model_for_new_data.ipynb notebook, which has slight adjustments compared to the Final-Model.ipynb notebook.
- It allows to import a new dataset, which is then merged with the original dataset, and goes through the same data cleaning, feature engineering and pre-processing as the other notebook.
- The train-test split is updated to use the entire original dataset for training and the new dataset as the test set.
- The win probabilities will be exported as "newdata_predictions.parquet"